In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xarray as xr

In [2]:
path = '/home/daniel/.local/share/WSJT-X - test/fmt.all'
starting_day = '2018-11-26'
frequency = 10706e6

In [3]:
def load_data(path, starting_day, frequency):
    with open(path) as f:
        data = [(s.replace('*', ' ').split()[0], float(s.split()[4]), float(s.split()[6])) for s in f.readlines()]
    timestamps = np.array([np.datetime64(starting_day + 'T' + d[0]) for d in data])
    # correct rollover
    timestamps += np.concatenate(([0], np.cumsum(np.diff(np.array(timestamps)) < np.timedelta64(0)))) * np.timedelta64(3600*24)
    ppb = np.array([(d[1]-500)/frequency*1e9 for d in data])
    snr = np.array([d[2] for d in data])
    data = xr.Dataset({'ppb' : (['time'], ppb), 'snr' : (['time'], snr)}, coords = {'time':timestamps})
    #data.attrs['units'] = 'ppb'
    #data.name = 'Frequency offset'
    return data

In [4]:
load_data(path, starting_day, frequency).to_netcdf('ppb.nc')